## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic']}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic']}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'UsageMetadata': {'title': 'UsageMetadata',
   'type': 'object',
   'properties': {'input_tokens': {'title':

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [6]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

为什么程序员总是把错误归咎于用户呢？因为他们从来不会犯错，只会写bug！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [7]:
chain.invoke({"topic": "程序员"})

AIMessage(content='为什么程序员总是在深夜工作？因为他们喜欢在黑暗中敲击键盘，仿佛是在黑客帝国中！', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 22, 'total_tokens': 71}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5c67061f-e76b-4d1b-9ee6-146bedf853bd-0')

### Batch
使用 .batch() 方法（同步）批量调用

In [8]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='为什么程序员总是懒惰？因为他们总是在努力减少代码行数，以便少写一点。', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 22, 'total_tokens': 61}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8be03fd1-3bfb-4601-b592-8b63edba95d9-0'),
 AIMessage(content='为什么产品经理总是在加班？\n因为他们总是在追求完美，而完美是永远无法实现的！', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 21, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-84198e02-10c1-4aa7-9b38-113df49e57d3-0'),
 AIMessage(content='测试经理在公司里开了个会，他说：“我们的测试工作必须尽可能完美，一定要做到百分之百的准确率！”\n\n一名员工问道：“那我们怎么知道我们的测试工作做的够好呢？”\n\n测试经理答道：“当你们测试完一个软件，用户反馈说‘这个软件简直就是神作！’，那就说明我们的工作做的够好了！”', response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 21, 'total_tokens': 150}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id=

In [9]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [10]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

为什么程序员总是觉得冷？
因为他们总是在byte。


笑话1:

为什么产品经理总是带着尺子？
因为他们要时刻保持产品的尺度。


笑话2:

为什么测试经理总是在周五下午发布bug修复的任务？

因为他们知道程序员们周末不会去开发新功能，只会专心修bug！😄




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [11]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

为什么程序员总是喜欢用咖啡因来提神工作？

因为他们觉得用 Java 来写代码，一杯咖啡是必须的！

### Async Invoke

In [12]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='为什么程序员总是带着电脑去厕所？\n因为他们觉得在那里才能真正的解决问题！😂', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 22, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f20841ef-c00f-4cbf-83eb-1d5413d21134-0')

### Async Batch

In [13]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='为什么程序员总是混淆“工作”和“生活”？\n\n因为他们总是把代码当成自己的生活，而把生活当成自己的代码。', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5210bade-2763-43c6-8695-a9edcc878eca-0'),
 AIMessage(content='为什么产品经理总是迟到？\n\n因为他们总是在等用户反馈！', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 21, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-db28a164-d641-4da9-bba0-3725f4b2f6bf-0'),
 AIMessage(content='为什么测试经理总是喜欢去钓鱼？\n\n因为他们喜欢测试水域的深度！', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 21, 'total_tokens': 58}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-239c2e42-8302-401a-ade8-31234f0b9af0-0')]